In [7]:
# default_exp display

# Displaying objects in notenooks

This module provides support for displaying objects, such as instance of GTSAM classes.

Graphviz and its python wrapper need to be installed. I did that with `conda install python-graphviz` which also installs graphviz, but these are installed by default on Google's colabs.


In [8]:
#export
import graphviz

In [9]:
#export
class show(graphviz.Source):
    """ Display an object with a dot method as a graph."""
    def __init__(self, obj):
        """Construct from object with 'dot' method."""
        # This small class takes an object, calls its dot function, and uses the 
        # resulting string to initialize a graphviz.Source instance. This in turn
        # has a _repr_mimebundle_ method, which then renders it in the notebook.
        super().__init__(obj.dot(False))